## Deploy SSD-VGG Model as Web Service on FPGA

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
import tensorflow as tf
import azureml

import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, os.path.abspath("../tfssd"))
from tfutil import endpoints
from finetune.model_saver import SaverVggSsd

### Restore AzureML workspace & register Model

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
from azureml.core.model import Model
from azureml.core.image import Image
from azureml.accel import AccelOnnxConverter
from azureml.accel import AccelContainerImage
from os.path import expanduser


model_ckpt_dir = expanduser("~/azml_ssd_vgg")
model_name = r'ssdvgg-fpga'
model_save_path = os.path.join(model_ckpt_dir, model_name)

# model_save_path should NOT exist prior to saving the model
not os.path.exists(model_save_path)

In [ ]:
with SaverVggSsd(model_ckpt_dir) as saver:
    saver.save_for_deployment(model_save_path)
    

In [ ]:
# Register model
registered_model = Model.register(workspace = ws,
                              model_path = model_save_path,
                              model_name = model_name)

print("Successfully registered: ", registered_model.name, registered_model.description, registered_model.version, '\n', sep = '\t')

### Convert inference model to ONNX format

In [ ]:
#Convert the TensorFlow graph to the Open Neural Network Exchange format (ONNX). 

input_tensor = saver.input_name_str
output_tensors_str = ",".join(saver.output_names)

# Convert model
convert_request = AccelOnnxConverter.convert_tf_model(ws, registered_model, input_tensor, output_tensors_str)

# If it fails, you can run wait_for_completion again with show_output=True.
convert_request.wait_for_completion(show_output=True)
converted_model = convert_request.result

print("\nSuccessfully converted: ", converted_model.name, converted_model.url, converted_model.version, 
      converted_model.id, converted_model.created_time, '\n')

### Create Docker Image

In [ ]:

image_config = AccelContainerImage.image_configuration()

# Image name must be lowercase
image_name = "{}-image".format(model_name)

image = Image.create(name = image_name,
                     models = [converted_model],
                     image_config = image_config, 
                     workspace = ws)
image.wait_for_creation(show_output=True)

# List the images by tag and get the detailed logs for any debugging.
print("Created AccelContainerImage: {} {} {}\n".format(image.name, image.creation_state, image.image_location))

### Deploy to the cloud

In [ ]:
#Create a new Azure Kubernetes Service

from azureml.core.compute import AksCompute, ComputeTarget

# Uses the specific FPGA enabled VM (sku: Standard_PB6s)
# Standard_PB6s are available in: eastus, westus2, westeurope, southeastasia
prov_config = AksCompute.provisioning_configuration(vm_size = "Standard_PB6s",
                                                    agent_count = 1, 
                                                    location = "westus2")

aks_name = 'aks-pb6-obj'

# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [ ]:
# Monitor deployment
aks_target.wait_for_completion(show_output=True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

In [ ]:
from azureml.core.webservice import Webservice, AksWebservice

# Set the web service configuration (for creating a test service, we don't want autoscale enabled)
# Authentication is enabled by default, but for testing we specify False
aks_config = AksWebservice.deploy_configuration(autoscale_enabled=False,
                                                num_replicas=1,
                                                auth_enabled = False)

aks_service_name ='fpga-aks-service'

aks_service = Webservice.deploy_from_image(workspace = ws,
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)


### Test the cloud service 

In [ ]:
# Using the grpc client in AzureML Accelerated Models SDK
from azureml.accel import PredictionClient

address = aks_service.scoring_uri
ssl_enabled = address.startswith("https")
address = address[address.find('/')+2:].strip('/')
port = 443 if ssl_enabled else 80
print(f"address={address}, port={port}, ssl={ssl_enabled}, name={aks_service.name}")

In [ ]:
# Initialize AzureML Accelerated Models client
client = PredictionClient(address=address,
                          port=port,
                          use_ssl=ssl_enabled,
                          service_name=aks_service.name)

In [ ]:
from tfutil import visualization
output_tensors = saver.output_names

### Visualize prediction using the deployed model

In [ ]:
import glob
import matplotlib as plt
import cv2

# Select an example image to test.  
# Default directory is the image_dir created in the Finetune VGG SSD notebook.

image_dir = expanduser("~/azml_ssd_vgg/JPEGImages")

im_files = glob.glob(os.path.join(image_dir, '*.jpg'))

im_file = im_files[0]


import azureml.accel._external.ssdvgg_utils as ssdvgg_utils

result = client.score_file(path=im_file, 
                           input_name=saver.input_name_str, 
                           outputs=output_tensors)

classes, scores, bboxes = ssdvgg_utils.postprocess(result, select_threshold=0.5)

plt.rcParams['figure.figsize'] = 15, 15
img = cv2.imread(im_file)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
visualization.plt_bboxes(img, classes, scores, bboxes)

### Clean up image (optional)

In [ ]:
#Delete your web service, image, and model (must be done in this order since there are dependencies).

#aks_service.delete()
#aks_target.delete()
#image.delete()
#registered_model.delete()
#converted_model.delete()